In [2]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model, load_model
from keras import backend as K

import tensorflow as tf

import os
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import aparent.visualization as vis

from aparent.predictor import *

import urllib
import urllib.request
import pickle
from time import sleep


Using TensorFlow backend.


In [3]:
#Load APADB Data and filter on targeted genes

#genes = ['RUNX1', 'CEBPA', 'GATA2', 'ANKRD26', 'DDX41', 'ETV6', 'PTEN', 'BRCA1', 'BRCA2', 'TP53', 'APC', 'ATM', 'PALB2', 'MSH2', 'MLH1', 'MSH6', 'PMS2', 'MUTYH']

polyadb_df = pd.read_csv('../../data/native_data/polyadb/processed_data/polyadb_processed.csv', sep=',')

#polyadb_df = polyadb_df.loc[polyadb_df['gene'].isin(genes)].reset_index(drop=True).copy()
polyadb_df = polyadb_df.loc[~polyadb_df['gene'].isna()].reset_index(drop=True).copy()
polyadb_df = polyadb_df.loc[polyadb_df['gene'] != 'na'].reset_index(drop=True).copy()

print('len(polyadb_df) = ' + str(len(polyadb_df)))


len(polyadb_df) = 190117


In [4]:
polyadb_df_minus = polyadb_df.query("strand == '-'").copy().reset_index(drop=True)
polyadb_df_plus = polyadb_df.query("strand == '+'").copy().reset_index(drop=True)

polyadb_df_minus = polyadb_df_minus.sort_values(by='pas_pos', ascending=False).copy().reset_index(drop=True)
polyadb_df_plus = polyadb_df_plus.sort_values(by='pas_pos', ascending=True).copy().reset_index(drop=True)

new_gene_id_list_plus = []
sitenum_list_plus = []
gene_id_dict = {}
for _, row in polyadb_df_plus.iterrows() :

    gene = str(row['gene'])

    if gene not in gene_id_dict :
        gene_id_dict[gene] = 0

    gene_id_dict[gene] += 1

    new_gene_id_list_plus.append(gene + "." + str(gene_id_dict[gene]))
    sitenum_list_plus.append(gene_id_dict[gene])

polyadb_df_plus['gene_id'] = new_gene_id_list_plus
polyadb_df_plus['sitenum'] = sitenum_list_plus

new_gene_id_list_minus = []
sitenum_list_minus = []
gene_id_dict = {}
for _, row in polyadb_df_minus.iterrows() :

    gene = str(row['gene'])

    if gene not in gene_id_dict :
        gene_id_dict[gene] = 0

    gene_id_dict[gene] += 1

    new_gene_id_list_minus.append(gene + "." + str(gene_id_dict[gene]))
    sitenum_list_minus.append(gene_id_dict[gene])

polyadb_df_minus['gene_id'] = new_gene_id_list_minus
polyadb_df_minus['sitenum'] = sitenum_list_minus

polyadb_df = pd.concat([polyadb_df_plus, polyadb_df_minus])

polyadb_df = polyadb_df.sort_values(by=['gene', 'sitenum'], ascending=True).reset_index(drop=True).copy()

In [5]:
polyadb_df.head()

,gene,gene_id,sitenum,num_sites,pas,seq,seq_ext,wide_seq,wide_seq_ext,rpm,total_rpm,ratio,pas_pos,cut_mode,chrom,strand,site_type
0,A1BG,A1BG.1,1,11,0,TTCTGCTAGGCTGGGCTAGTATCTTCCATGGCAAGATACTCAAACT...,TAGGGAATATAACTTTTAAAATGCGAAAGTATATTATACATATTGC...,AGTTTGGCTAGCATTTCAAAATGGATAGGGAATATAACTTTTAAAA...,GCTTTCAAAATTCCTGATTAGCTTACAATTAATTGCAAATAACTTC...,1.564234,24.390222,0.064134,58859852,58859830,chr19,-,Intron
1,A1BG,A1BG.2,2,11,-1,AGGTGCCGCTACCGCTCCTGGGTGCCCCACACCTTCGAATCGGAGC...,CGGCGAACCTCGAGCTGATCTTCGTGGGGCCCCAGCACGCCGGCAA...,AAGACGGTCCGCACCCCCGGGGCCGCGGCGAACCTCGAGCTGATCT...,TCCCCGACGTCACCTTCGAGCTGCTGCGCGAGGGCGAGACGAAGGC...,1.740036,24.390222,0.071342,58858744,58858719,chr19,-,Internal_exon
2,A1BG,A1BG.3,3,11,0,CCGGGGTCTCTGGGGGGCTGGAGAAGCCTCCCTCATTCCTCCCAGG...,AGTGCCGGGGATTCTGGACTGGCTCCCTCCCCTCCTGTTGCAGCAC...,CCAGGGTGCTGTTGGTGTCCTCAGAAGTGCCGGGGATTCTGGACTG...,CACCATTTCTCTTTTCTCCCGTTGTCTAGAAAGCTGATGCAGCCGC...,11.529986,24.390222,0.472730,58858249,58858220,chr19,-,3_most_exon
3,A1BG,A1BG.4,4,11,4,GAAGATGCGTATCAGTATTATCAACCAATAGTTGATGAATTGTGTT...,ACCACTGCCGAGTTTCTGCCTGGAGCAGGGCCTCAAGGCCCTGGCA...,CCATCTGACATCTGATGCCTTCCCAACCACTGCCGAGTTTCTGCCT...,GATTCCTCTGTGTTCTTTCCCATTATCCCTCAGCTCTGAGTCCTCT...,1.386686,24.390222,0.056854,58857215,58857210,chr19,-,3_most_exon
4,A1BG,A1BG.5,5,11,4,AACGAATTTGCTAGTGATCTGGTTTACTGCCTTAGTAATATCTAGT...,GAAGATGCGTATCAGTATTATCAACCAATAGTTGATGAATTGTGTT...,CAGGGCCTCAAGGCCCTGGCACACAGAAGATGCGTATCAGTATTAT...,CCATCTGACATCTGATGCCTTCCCAACCACTGCCGAGTTTCTGCCT...,1.174863,24.390222,0.048169,58857165,58857142,chr19,-,3_most_exon


In [6]:

polyadb_df['start'] = polyadb_df['pas_pos'] - 50
polyadb_df['end'] = polyadb_df['pas_pos'] + 100

polyadb_bed = polyadb_df[['chrom', 'start', 'end', 'gene', 'gene_id', 'strand']].copy().reset_index(drop=True)

polyadb_bed.head()

polyadb_bed.to_csv("polyadb_coordinates.bed", sep='\t', header=False, index=False)


In [8]:

snp_df = pd.read_csv("APApeak_Phenotype_GeneLocAnno.Nuclear_permResBH.txt", sep=' ')
snp_df = snp_df.query("dist <= 100 and dist >= -100").copy().reset_index(drop=True)


In [32]:

def get_html(url_str, retries=3) :
    if retries <= 0 :
        print("Could not open '" + url_str + "'")
        return ''
    
    try :
        html = ''
        with urllib.request.urlopen(url_str) as response:
            html = str(response.read().decode("utf-8"))
        return html
    except urllib.request.URLError :
        return get_html(url_str, retries=retries-1)

'''
def extract_snp_alleles_from_html(html) :
    html_no_wspace = html.replace(' ', '').strip()
    if html_no_wspace == '' or not ("VariationType</dt>\n<dd>\n\n" in html_no_wspace) :
        return [], "N/A"
    
    snp_type = html_no_wspace.split("VariationType</dt>\n<dd>\n\n")[1].split("\n")[0]
    if snp_type != 'SNV' :
        return [], snp_type
    
    allele_str = html_no_wspace.split("Alleles</dt>\n<dd>\n\n\n")[1].split("\n")[0]
    alleles = allele_str.split("/")
    alleles = [allele.split(">")[1] for allele in alleles]
    
    return alleles, snp_type
'''
def extract_snp_alleles_from_html(html) :
    html_no_wspace = html.replace(' ', '').strip()
    if html_no_wspace == '' or not ("VariationType</dt>\n<dd>\n\n" in html_no_wspace) :
        return "N/A", -1, ["X", "X"], "N/A"
    
    snp_type = html_no_wspace.split("VariationType</dt>\n<dd>\n\n")[1].split("\n")[0]
    if snp_type != 'SNV' :
        return "N/A", -1, ["X", "X"], snp_type
    
    chr_str = html_no_wspace.split("<td>GRCh37")[1].split("</td>")[0].split("hr")[1]
    allele_str = html_no_wspace.split("<td>GRCh37")[1].split("<td>")[1].split("</td>")[0].split(":g.")[1]
    
    pos = allele_str[:-3]
    
    alleles = allele_str[-3:]
    
    return chr_str, pos, [alleles[0], alleles[2]], snp_type


In [33]:
snp_df

,pid,nvar,shape1,shape2,dummy,sid,dist,npval,slope,ppval,bpval,bh,chr,start,end,ref_allele,var_allele
0,10:75785548:75785639:VCL_intron_-_peak17074,77,1.001460,12.80610,37.3656,rs79708660,-83.0,2.985400e-03,0.621044,0.073926,7.700990e-02,5.772289e-01,chr10,75785466,75785467,G,>
1,10:97365695:97365785:ALDH18A1_utr3_+_peak18185,130,0.865116,13.64120,38.0359,rs11188395,-74.0,7.323760e-03,0.240176,0.213786,2.140500e-01,7.403186e-01,chr10,97365622,97365623,G,>
2,10:122612555:122612647:WDR11_intron_-_peak19609,183,0.982916,21.45470,40.4422,rs10886786,20.0,2.571090e-02,-0.364328,0.510490,5.166290e-01,8.855123e-01,chr10,122612576,122612577,A,>
3,10:124693568:124693659:C10orf88_intron_+_peak1...,87,0.947541,15.06850,44.6531,rs7904973,18.0,1.281670e-08,1.255120,0.000999,3.497940e-07,1.306029e-04,chr10,124693587,124693588,G,>
4,10:129896154:129896244:MKI67_utr3_+_peak19885,166,1.058520,31.20620,38.3797,rs7347,77.0,1.266780e-06,-0.861094,0.000999,1.133580e-04,1.410819e-02,chr10,129896232,129896233,A,>
5,10:129897408:129897490:MKI67_utr3_+_peak19887,162,1.011290,35.14380,41.6625,rs4750935,-95.0,1.320780e-05,-0.931377,0.000999,7.273800e-04,5.289024e-02,chr10,129897314,129897315,C,>
6,11:8413591:8413681:STK33_utr3_+_peak20663,167,1.070860,30.72990,37.6406,rs7934876,9.0,1.743360e-05,-0.950998,0.000999,1.392250e-03,8.473346e-02,chr11,8413601,8413602,G,>
7,11:35017434:35017517:PDHX_utr3_-_peak21732,185,1.074670,22.18660,35.0625,rs7808,-72.0,1.031940e-02,0.748673,0.348651,3.578300e-01,8.248687e-01,chr11,35017363,35017364,C,>
8,11:45834481:45834568:SLC35C1_utr3_-_peak21982,94,1.056670,19.60730,38.0875,rs79173599,40.0,2.871760e-02,-1.007070,0.560440,5.435550e-01,8.944406e-01,chr11,45834522,45834523,G,>
9,11:65120366:65120451:DPF2_utr3_-_peak23006,60,1.007280,14.85350,38.2250,rs5858,24.0,6.678680e-08,0.779964,0.000999,6.710120e-06,1.447904e-03,chr11,65120391,65120392,T,>


In [34]:

chroms = []
starts = []
ends = []
ref_alleles = []
var_alleles = []

i = 0
for _, row in snp_df.iterrows() :
    
    if i % 50 == 0 :
        print("Crawling dbSNP page " + str(i) + "...")
    
    url_str = 'https://www.ncbi.nlm.nih.gov/snp/' + row['sid'].split("-")[0]
    
    chrom, pos, alleles, snp_type = extract_snp_alleles_from_html(get_html(url_str, retries=5))
    
    chroms.append("chr" + chrom)
    starts.append(int(pos))
    ends.append(int(pos) + 1)
    ref_alleles.append(alleles[0])
    var_alleles.append(alleles[1])
    
    i += 1

snp_df['chr'] = chroms
snp_df['start'] = starts
snp_df['end'] = ends
snp_df['ref_allele'] = ref_alleles
snp_df['var_allele'] = var_alleles


Crawling dbSNP page 0...
Crawling dbSNP page 50...
Crawling dbSNP page 100...
Crawling dbSNP page 150...
Crawling dbSNP page 200...


In [36]:

snp_df['strand'] = snp_df['pid'].apply(lambda x: "+" if "_+_" in x else "-")
snp_df['gene'] = snp_df['pid'].apply(lambda x: x.split(":")[3].split("_")[0])


In [39]:

snp_df = snp_df[['chr', 'start', 'end', 'gene', 'sid', 'strand', 'pid', 'nvar', 'shape1', 'shape2', 'npval', 'slope', 'ref_allele', 'var_allele']]


In [45]:

snp_df = snp_df.loc[snp_df['chr'] != 'chrN/A'].copy().reset_index(drop=True)


In [46]:

snp_df.to_csv("elife_snp_catalog.bed", sep='\t', header=False, index=False)


In [47]:

!bedtools intersect -a polyadb_coordinates.bed -b elife_snp_catalog.bed -wa -wb > polyadb_elife_intersect.bed


In [5]:

snp_df = pd.read_csv("polyadb_elife_intersect.bed", sep='\t', error_bad_lines=False, names=['chrom', 'start', 'end', 'gene', 'gene_id', 'strand', 'chr_2', 'snp_pos', 'end_2', 'gene_2', 'snp_id', 'strand_2', 'pid', 'nvar', 'shape1', 'shape2', 'p_value', 'slope', 'snp_ref', 'snp_var'])
snp_df = snp_df.query("gene == gene_2").copy().reset_index(drop=True)

print(len(snp_df))
print(snp_df.head())


128
   chrom      start        end    gene    gene_id strand  chr_2    snp_pos  \
0  chr18   19243930   19244080   ABHD3    ABHD3.5      -  chr18   19243976   
1  chr18   19243899   19244049   ABHD3    ABHD3.6      -  chr18   19243976   
2   chr1  226352446  226352596   ACBD3    ACBD3.4      -   chr1  226352498   
3   chr7   87828152   87828302  ADAM22  ADAM22.11      +   chr7   87828204   
4   chr2   61418261   61418411   AHSA2    AHSA2.9      +   chr2   61418343   

       end_2  gene_2      snp_id strand_2  \
0   19243977   ABHD3  rs11665314        +   
1   19243977   ABHD3  rs11665314        +   
2  226352499   ACBD3   rs2306120        +   
3   87828205  ADAM22  rs10268218        -   
4   61418344   AHSA2    rs777592        -   

                                            pid  nvar    shape1   shape2  \
0   18:19243932:19244042:ABHD3_utr5_+_peak60420    70  0.949334  15.4727   
1   18:19243932:19244042:ABHD3_utr5_+_peak60420    70  0.949334  15.4727   
2   1:226352490:226352592:AC

In [6]:

polyadb_df_sel = polyadb_df[['wide_seq_ext', 'gene_id', 'pas', 'pas_pos', 'cut_mode', 'site_type', 'rpm', 'total_rpm', 'ratio']].copy().reset_index(drop=True)

snp_df = snp_df.join(polyadb_df_sel.set_index('gene_id'), on='gene_id').copy().reset_index(drop=True)
snp_df = snp_df.drop_duplicates(subset=['snp_id'], keep='first')

print(len(snp_df))
print(snp_df.head())


85
   chrom      start        end    gene    gene_id strand  chr_2    snp_pos  \
0  chr18   19243930   19244080   ABHD3    ABHD3.5      -  chr18   19243976   
2   chr1  226352446  226352596   ACBD3    ACBD3.4      -   chr1  226352498   
3   chr7   87828152   87828302  ADAM22  ADAM22.11      +   chr7   87828204   
4   chr2   61418261   61418411   AHSA2    AHSA2.9      +   chr2   61418343   
6   chr1   46035649   46035799  AKR1A1   AKR1A1.3      +   chr1   46035674   

       end_2  gene_2    ...    snp_ref snp_var  \
0   19243977   ABHD3    ...          T       G   
2  226352499   ACBD3    ...          T       G   
3   87828205  ADAM22    ...          C       A   
4   61418344   AHSA2    ...          G       C   
6   46035675  AKR1A1    ...          T       C   

                                        wide_seq_ext  pas    pas_pos  \
0  TTGGAGACAGTTATTCACCATGTACACAGCCTGTACCCTTCTGCTC...    0   19243980   
2  TACTAAGAAGGGGAGGTAACTTTTCTTGTTATTCCTTACAGGAGAG...    3  226352496   
3  AGAAGTTA

In [7]:

def get_snp_pos(row) :
    if row['strand'] == '+' :
        return row['snp_pos'] - row['pas_pos'] + 175 - 1
    else :
        return row['pas_pos'] - row['snp_pos'] + 175

def get_snp_ref(row) :
    if row['strand'] == '+' :
        return row['snp_ref']
    else :
        if row['snp_ref'] == 'A' :
            return 'T'
        if row['snp_ref'] == 'C' :
            return 'G'
        if row['snp_ref'] == 'G' :
            return 'C'
        if row['snp_ref'] == 'T' :
            return 'A'

def get_snp_var(row) :
    if row['strand'] == '+' :
        return row['snp_var']
    else :
        if row['snp_var'] == 'A' :
            return 'T'
        if row['snp_var'] == 'C' :
            return 'G'
        if row['snp_var'] == 'G' :
            return 'C'
        if row['snp_var'] == 'T' :
            return 'A'

def get_var_seq(row) :
    ref_seq = row['wide_seq_ext']
    var_seq = ref_seq[:row['rel_snp_pos']] + row['rel_snp_var'] + ref_seq[row['rel_snp_pos']+1:]
    
    return var_seq

snp_df['rel_snp_pos'] = snp_df.apply(get_snp_pos, axis=1)
snp_df['rel_snp_ref'] = snp_df.apply(get_snp_ref, axis=1)
snp_df['rel_snp_var'] = snp_df.apply(get_snp_var, axis=1)

snp_df['wide_seq_ext_var'] = snp_df.apply(get_var_seq, axis=1)


In [61]:

#Save final apaQTL dataframe
snp_df.to_csv("polyadb_elife_SNPs.csv", sep='\t')
